In [ ]:
import pandas as pd

In [8]:
train = pd.read_csv("data/train.csv", parse_dates=["first_active_month"])
test = pd.read_csv("data/test.csv", parse_dates=["first_active_month"])
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [9]:
historical_transactions = pd.read_csv("data/historical_transactions.csv")
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [ ]:
# Calculate number of historical transactions by card_id
purchase_stats = historical_transactions.groupby('card_id')['purchase_amount'].agg([
    'mean',
    'std',
    'min',
    'max',
    'count',
    'sum'
]).reset_index()

# Rename columns for clarity
purchase_stats. columns = ['card_id', 'avg_purchase', 'std_purchase', 
                        'min_purchase', 'max_purchase', 'num_transactions', 'total_spent']


# Merge with training and test set
train = train.merge(purchase_stats, on="card_id", how="left")
test = test.merge(purchase_stats, on="card_id", how="left")

train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,transactions_count,avg_purchase,std_purchase,min_purchase,max_purchase,num_transactions,total_spent
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,260,-0.638341,0.212139,-0.739395,2.258395,260,-165.968739
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,350,-0.600018,0.384967,-0.742400,4.630299,350,-210.006336
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,43,-0.678311,0.087380,-0.730138,-0.145847,43,-29.167391
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,77,-0.642745,0.261624,-0.740897,1.445596,77,-49.491364
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,133,-0.366073,1.352094,-0.746156,7.193041,133,-48.687656
